In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# !pip install --upgrade segmentation_models_pytorch

In [2]:
import os
import shutil
import pathlib
import gc
import pandas as pd
import ttach as tta
from pathlib import Path
from PIL import Image
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset
from torch.optim.swa_utils import AveragedModel, SWALR

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import timm

In [3]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
seed_everything(42)

In [5]:
ensemple_type = "No" #"No" #"mine"

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [7]:
import torchvision.transforms as T


class ContrailDataset:
    def __init__(self, df, transform=None, normalize=False):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.normalize=normalize
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        

        # label_cls = 1 if label.sum() > 0 else 0
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
            
#         return torch.tensor(image), torch.tensor(label)
    
        if self.normalize:
            image = self.normalize_image(torch.tensor(image))
            return image, torch.tensor(label)
        else:
            return torch.tensor(image), torch.tensor(label)
    
    

In [8]:
###TImmunetplusplus model Nirjhar

n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(
            decoder_channels[n_blocks-1],
            cfg["num_classes"], 
            kernel_size=(3, 3),
            stride=(1, 1), 
            padding=(1, 1)
        )

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model1(cfg):
    model = TimmSegModel(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


#### Model 2 Nirjhar

n_blocks =4
class TimmSegModel2(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel2, self).__init__()

        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model2(cfg):
    model = TimmSegModel2(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


##################

n_blocks = 4

class TimmSegModel3(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel3, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], cfg['num_classes'], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
    
def load_model3(cfg):
    model = TimmSegModel3(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model



class TimmSegModel4(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel4, self).__init__()

        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 512, 512))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
def load_model4(cfg):
    model = TimmSegModel4(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model







## Rohit Model 
class Net_R(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        if cfg['model_name'] == 'Unet':
            self.model = smp.Unet(
                encoder_name=cfg["backbone"],     
                encoder_weights=None,   
                in_channels=3,  
                classes=cfg["num_classes"],
                activation=None
            )
            
        elif cfg['model_name'] == 'FPN':
            self.model = smp.FPN(
                encoder_name=cfg["backbone"],     
                encoder_weights=None,   
                in_channels=3,  
                classes=cfg["num_classes"],
                activation=None
            )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask
    
    
def load_modelr1(cfg):
    model = Net_R(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model


def load_modelr1(cfg):
    model = Net_R(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model


# def load_modelr3(cfg):
# #     weight = f"{cfg.architecture.pretrained_weights}"
#     model = Net_R(cfg)
#     model = torch.nn.DataParallel(model).cuda()
    
#     d =  torch.load(cfg['model_pth'], map_location="cpu")
#     if "model" in d:
#         model_weights = d["model"]
#     else:
#         model_weights = d
#     try:
#         model.load_state_dict(model_weights, strict=True)
#     except Exception as e:
#         print("removing unused pretrained layers")
#         for layer_name in re.findall("size mismatch for (.*?):", str(e)):
#             model_weights.pop(layer_name, None)
#         model.load_state_dict(model_weights, strict=False)

#     print(f"Weights loaded from: {cfg.architecture.pretrained_weights}")



#     if idx in [11, 12]:
#         model = TimmSegModelR1(cfg)
#         model = torch.nn.DataParallel(model).cuda()
#         model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
#     else:
#         model = Net(cfg)    
#         model = torch.nn.DataParallel(model).cuda()
#         model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])

class TimmSegModelR1(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModelR1, self).__init__()

        self.n_blocks = 4
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:self.n_blocks+1],
                decoder_channels=decoder_channels[:self.n_blocks],
                n_blocks=self.n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[self.n_blocks-1], 
                      cfg['num_classes'],
                      kernel_size=(3, 3),
                      stride=(1, 1),
                      padding=(1, 1)
            ),
            nn.UpsamplingBilinear2d(scale_factor=1)
        )

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:self.n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features


    
def load_modelr2(cfg):
    model = TimmSegModelR1(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model

### Ioannis

In [9]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

def get_transform2(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_LINEAR),
    ], p=1.0)
    return transform




In [10]:
val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")
val_df_full = val_df.copy()

val_dups = np.load("../../input/data_utils/dups_val.npy")

val_dups = [int(val_id) for val_id in val_dups]

In [11]:
# val_df = val_df.loc[~val_df['id'].isin(val_dups)].reset_index(drop=True)
print(val_df.shape, val_df_full.shape)

(1856, 4) (1856, 4)


In [87]:
CFGS1 = [
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01",
#         'model_func': load_modelr1,
#         'tta': False,
#         'normalize': False
#     }, 

    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s2/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_02_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_03_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s2_swa/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_04_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s8_swa/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_05_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_s9_swa/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_06_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, 
    
    
#     {
#         'model_name': 'FPN',
#         'backbone': 'efficientnet-b6',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_swa/FPN/efficientnet-b6-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_07_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, # nir_07_tta hvflip
    
 
#     {
#         'model_name': 'FPN',
#         'backbone': 'efficientnet-b6',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output1/exp_10_swa_s2/FPN/efficientnet-b6-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_07_tta",
#         'model_func': load_modelr1,
#         'tta': True,
#         'normalize': False
#     }, # nir_07_tta
    

    
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01_tta",
#         'model_func': load_model4,
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01_tta",
#         'model_func': load_model4,
#         'tta': False,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02_tta",
#         'model_func': load_model2,
#         'tta': True,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04_tta", 
#         'model_func': load_model1,
#         'tta': True,
#         'normalize': False
#     }, 
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/fullfinetune-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2cv670-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_05_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ema_ema-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_06_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/pseudostage2-effb8modelv2fold0_tf_efficientnet_b8_best_ctrl_epoch-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
    
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/pseudostage2-effb8modelv2fold0_tf_efficientnet_b8_best_ctrl_epoch-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07", 
#         'model_func': load_model3,
#         'tta': False,
#         'normalize': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01",
#         'model_func': load_model4,
#         'tta': False,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02",
#         'model_func': load_model2,
#         'tta': False,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03", 
#         'model_func': load_model3,
#         'tta': False,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04", 
#         'model_func': load_model1,
#         'tta': False,
#         'normalize': False
#     }, 
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/fullfinetune-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2cv670-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_05", 
#         'model_func': load_model3,
#         'tta': False,
#         'normalize': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/swa-pseudostage3-effb8modelv2fold0v2_tf_efficientnet_b8_best_ctrl_epoch-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07_tta", 
#         'model_func': load_model3,
#         'tta': True,
#         'normalize': False
#     },
    
    {
        'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b8',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/fullfinetune-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2cv670-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_08_tta", 
        'model_func': load_model3,
        'tta': True,
        'normalize': False
    },
    
    

    
    

    
]

In [88]:
import cv2
import numpy as np

# img = cv2.imread('some_image.png',0)
# closing = cv2.morphologyEx(img, cv.MORPH_CLOSE, kernel)

In [89]:
final_preds1 = []

for idx, cfg in enumerate(CFGS1):   

#     if idx <= 10:
#         continue
        
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    val_transform2 = get_transform2(cfg['img_size'])

    valid_dataset = ContrailDataset(val_df, transform=val_transform, normalize=False)  
    if cfg['normalize'] and ("ioa_" in cfg['call_sign']):
        valid_dataset = ContrailDataset(val_df, transform=val_transform2, normalize=True)  
#         valid_dataset = ContrailDataset(val_df, transform=val_transform, normalize=True)  


    
    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )
    
    
    model = cfg['model_func'](cfg)        
      
    if cfg['tta']:
        if "roh_" in cfg['call_sign']:
            model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
#             model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
        else:
            model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
#             model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
#             model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')


    
    model.to(device)
    model.eval()
    
    
#     model_base.to(device)
#     model_base.eval()
    
    
    preds = []
    masks_ = []      
        
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
#             masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 


        masks_.append(torch.squeeze(masks, dim=1))
        with torch.inference_mode():
            images = torch.nn.functional.interpolate(images, size=cfg['img_size'][0], mode='nearest')
            pred = model(images).sigmoid()   
                
            pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest')
            preds.append(torch.squeeze(pred, dim=1))
        
            
            
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
        
    print(model_preds.shape)
        
#     model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
#     model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
    # save
#     torch.save(model_preds, f"../../output/final_preds/{cfg['call_sign']}.pt")    
#     torch.save(model_preds, f"../../output/final_preds/{cfg['call_sign']}_full.pt")    

#     if idx == 6:
#     torch.save(model_masks, f'../../output/final_preds/val_masks.pt')

    np.save(f"../../output/final_preds/{cfg['call_sign']}", model_preds.detach().cpu().numpy())
#     if idx == 0:
#         np.save(f"../../output/final_preds/val_masks", model_masks.detach().cpu().numpy())

    
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
    final_preds1.append(model_preds)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()

{'model_name': 'Unet++', 'backbone': 'tf_efficientnet_b8', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/nirjhar/fullfinetune-effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2cv670-00.bin', 'threshold': 0.79, 'call_sign': 'nir_08_tta', 'model_func': <function load_model3 at 0x7f3754ecb130>, 'tta': True, 'normalize': False}


  0%|          | 0/58 [00:00<?, ?it/s]

torch.Size([1856, 256, 256])
0.6744085 0.76


In [91]:
# call_signs1 = [
# #     "roh_01_tta", # can be removed
# #     "roh_02_tta", 
# #     "roh_03_tta", 

#     "roh_04_tta", 
#     "roh_05_tta",
#     "roh_06_tta",
#     "nir_01_tta",
    
# #     "nir_02_tta",
# #     "nir_03_tta",
# #     "nir_04_tta",
#     "nir_05_tta", # remove this from ensemble
# #     "nir_06_tta", # use this
# #     "nir_07_tta",
# #     "roh_05_tta",
    
# #     "exp32_Unet-tf_efficientnet_b8_soup_preds_0.6663",
    
#     "121", 
#     "122", 
#     "125", 
#     "127", 
#     "131"  
# ] 


call_signs1 = [

    "nir_08_tta",
    "nir_07_tta", 
    "roh_02_tta", 
    "nir_01",
    "nir_05",
    "162_tta",
    "166_tta",
    "121_tta",
    "162",
    "125"
    
] 

model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).flatten() 

preds = []

for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    if idx == 0:
        wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.npy")).flatten()
    else:
        wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).flatten()

    preds.append(wt)
    print(wt.shape)
    
    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
    
    

    
# preds.append(x1)
    
final_preds = preds
final_preds = torch.stack(final_preds).mean(dim=0)
score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 

print("0.5 TH Score: ", score)


best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] :
    if threshold < 0.35 or threshold > 0.51:
        continue
    
    score = dice_coef(model_masks, final_preds, thr=threshold).cpu().detach().numpy() 
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold
        
print(best_dice_score, best_threshold)

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([121634816])
0.67340404
torch.Size([121634816])
0.6766576
torch.Size([121634816])
0.67922837
torch.Size([121634816])
0.6578587
torch.Size([121634816])
0.6704452
torch.Size([121634816])
0.6818373
torch.Size([121634816])
0.67855626
torch.Size([121634816])
0.68600667
torch.Size([121634816])
0.6730113
torch.Size([121634816])
0.65260977
0.5 TH Score:  0.69378453
0.6959636 0.36


In [86]:
call_signs1 = [
#     "roh_04_tta", 
#     "roh_05_tta", 
#     "roh_05_tta1", 
#     "roh_05_tta2", 
#     "roh_05_tta3", 
#     "roh_05_tta4", 
    
#     "roh_07_tta"
    
    
# #     "roh_08_tta",
#     "roh_02_tta", 
#     "nir_06_tta",
#     "162_tta",
#     "166_tta",
#     "121_tta",
    
    "nir_07_tta", 
    "roh_02_tta", 
    "nir_01",
    "nir_05",
    "162_tta",
    "166_tta",
    "121_tta",
    "162",
    "125"
] 

model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).flatten() 

preds = []

for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    if idx == 0: 
        wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.npy")).flatten()   
    else:
        wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).flatten()   
#     if idx != 0:
    preds.append(wt)
    print(wt.shape)

    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
    
    

    
# preds.append(x1)
    
final_preds = preds
final_preds = torch.stack(final_preds).mean(dim=0)
score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 

print("0.5 TH Score: ", score)


  0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([121634816])
0.67922837
torch.Size([121634816])
0.6578587
torch.Size([121634816])
0.6704452
torch.Size([121634816])
0.6818373
torch.Size([121634816])
0.67855626
torch.Size([121634816])
0.68600667
torch.Size([121634816])
0.6730113
torch.Size([121634816])
0.65260977
0.5 TH Score:  0.6945755


In [67]:
# x1 = final_preds  0.6951733

In [94]:
call_signs1 = [
# #     "roh_01_tta", # can be removed
#     "roh_02_tta",    
# #     "roh_04_tta", 
#     "roh_05_tta", 
#     "nir_01_tta",
#     "nir_05_tta", # remove this from ensemble
# #     "nir_06_tta",

    
     #roh_01_tta roh_02_tta, roh_07_tta, roh_06, roh_07
    # roh_03_tta, roh_06_tta, roh_03, roh_05, nir_06_tta, nir_01, nir_02
    # roh_04_tta, roh_01, nir_02_tta, 131
    # roh_02, nir_03_tta, nir_04_tta, nir_05_tta, nir_06, 160
    
#     "roh_01_tta", # can be removed
#     "roh_03_tta", 
#     "roh_04_tta", 
#     "roh_05_tta",
#     "roh_06_tta",
#     "roh_07_tta",
    
#     "roh_01", # can be removed
#     "roh_02", 
#     "roh_03", 
#     "roh_04", 
#     "roh_05",
#     "roh_06",
#     "roh_07",
    
#     "nir_01_tta",
#     "nir_02_tta", # remove this from ensemble
#     "nir_03_tta",
#     "nir_04_tta",
#     "nir_05_tta",
#     "nir_06_tta",
    
        
#     "nir_01",
#     "nir_02", # remove this from ensemble
#     "nir_03",
#     "nir_04",
#     "nir_06",
    
    
#     "121", 
#     "122", 
#     "125", 
#     "127", 
#     "131",
    
#     "160", 
#     "162", 
#     "166", 
#     "160_tta", 
    
    
#     "roh_01_tta", ##
    #"roh_03_tta", ##
#     "roh_04_tta", ##
#     "roh_05_tta",###
#     "roh_06_tta", ##
#     "roh_01", ##
    ##"roh_07_tta", ###
    
#     "122", ###
    
    
    
#     "162",
#     "125",
#     "nir_01",
#     "roh_02_tta", 
#     "nir_05",
#     "162_tta",
#     "166_tta",
#     "121_tta",
    

    "nir_08_tta",
    "nir_07_tta",
    "roh_02_tta", 
    "nir_01",
    "nir_05",
    "162_tta",
    "166_tta",
    "121_tta",
    "162",
    "125"
    
#     "roh_08_tta",
# #     "roh_02_tta", 
#     "nir_06_tta",
#     "162_tta",
#     "166_tta",
#     "121_tta",
    
    
] 

model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).flatten().cuda() 

preds = []

for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    if idx == 0:
        wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.npy")).flatten().cuda()  
    else:
        wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).flatten().cuda()  
    preds.append(wt)
    print(wt.shape)
    
    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
    
    
# # preds1.append(preds2)
    
final_preds = preds
# final_preds = torch.stack(final_preds).mean(dim=0)
# score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 

# print("0.5 TH Score: ", score)


# best_threshold = 0.0
# best_dice_score = 0.0
# for threshold in [i / 100 for i in range(101)] :
#     if threshold < 0.35 or threshold > 0.51:
#         continue
    
#     score = dice_coef(model_masks, final_preds, thr=threshold).cpu().detach().numpy() 
#     if score > best_dice_score:
#         best_dice_score = score
#         best_threshold = threshold
        
# print(best_dice_score, best_threshold)

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([121634816])
0.67340404
torch.Size([121634816])
0.6766576
torch.Size([121634816])
0.67922837
torch.Size([121634816])
0.6578587
torch.Size([121634816])
0.6704452
torch.Size([121634816])
0.6818373
torch.Size([121634816])
0.67855626
torch.Size([121634816])
0.68600667
torch.Size([121634816])
0.6730113
torch.Size([121634816])
0.65260977


In [95]:
def boost_dice(params, TH=None):    
    
    for index, val in enumerate(params.keys()):
        if index == 0:            
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
    if TH:
        best_threshold = TH
        preds1 = (preds).double()
        best_score = dice_coef(model_masks, preds1, thr=TH).cpu().detach().numpy() # get_score(labels, preds) 
    else:
        best_threshold = 0.0
        best_score = 0.0
        for threshold in [i / 100 for i in range(101)] :              
            if threshold < 0.3 or threshold > 0.65:
                continue

            preds1 = ((preds)>threshold).double()            
            score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
            if score > best_score:
                best_score = score
                best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [98]:
def objective(trial):
    TH = 0.37
    
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)
        
#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)
#     params[f"w1"] = 0.15614990824242336
#     params[f"w2"] = 0.3684759350037226
#     params[f"w3"] = 0.2047525215621725
#     params[f"w4"] = 0.36485377740463365
#     params[f"w5"] = 0.5716506827395362
#     params[f"w6"] = 0.84301760289886
#     params[f"w7"] = 0.5944703175254541
#     params[f"w8"] = 0.8096022495041699

        
        
        
        
    score, best_threshold  = boost_dice(params, TH=TH)
    params['threshold'] = best_threshold
    if not TH:
        print(params)
        
    return score

In [99]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)
best_params = study.best_params

[I 2023-07-22 19:27:00,671] A new study created in memory with name: no-name-d46b179b-76a5-4eab-b3f4-38b2421f51e9
[I 2023-07-22 19:27:00,714] Trial 0 finished with value: 0.6885680556297302 and parameters: {'w1': 0.7495341996699179, 'w2': 0.8286574593175949, 'w3': 0.21553649243935624, 'w4': 0.750715390734391, 'w5': 0.757431246681187, 'w6': 0.10137680050612585, 'w7': 0.3525492843984098, 'w8': 0.482363029179086, 'w9': 0.04314683924027052, 'w10': 0.39677932885695166}. Best is trial 0 with value: 0.6885680556297302.
[I 2023-07-22 19:27:00,755] Trial 1 finished with value: 0.6972655653953552 and parameters: {'w1': 0.12397762944196311, 'w2': 0.29544893894244284, 'w3': 0.523552753459589, 'w4': 0.8621444276114175, 'w5': 0.8954650162718661, 'w6': 0.028584289971692933, 'w7': 0.6418343141072657, 'w8': 0.4335290581372766, 'w9': 0.9839979457743863, 'w10': 0.8290666211096039}. Best is trial 1 with value: 0.6972655653953552.
[I 2023-07-22 19:27:00,796] Trial 2 finished with value: 0.6976138353347778 

[I 2023-07-22 19:27:01,963] Trial 20 finished with value: 0.6985953450202942 and parameters: {'w1': 0.3831096475162926, 'w2': 0.46140309617384856, 'w3': 0.48572526747844724, 'w4': 0.7723306944708963, 'w5': 0.20793839801794767, 'w6': 0.994807743320655, 'w7': 0.7378116364733561, 'w8': 0.609031695145383, 'w9': 0.6094180682426296, 'w10': 0.41331776444464785}. Best is trial 20 with value: 0.6985953450202942.
[I 2023-07-22 19:27:02,044] Trial 21 finished with value: 0.6985260844230652 and parameters: {'w1': 0.3939198257519671, 'w2': 0.4744741052960168, 'w3': 0.4626401830329999, 'w4': 0.7799911218979938, 'w5': 0.20835561664468602, 'w6': 0.9731837154706703, 'w7': 0.7031185431236918, 'w8': 0.6159247210049343, 'w9': 0.5844920662337281, 'w10': 0.35381499014296935}. Best is trial 20 with value: 0.6985953450202942.
[I 2023-07-22 19:27:02,125] Trial 22 finished with value: 0.698566198348999 and parameters: {'w1': 0.3799395982400446, 'w2': 0.4479892151231819, 'w3': 0.4655319069044374, 'w4': 0.7948587

[I 2023-07-22 19:27:03,658] Trial 41 finished with value: 0.6996533274650574 and parameters: {'w1': 0.10821952155820458, 'w2': 0.004379100010970249, 'w3': 0.8103840367500296, 'w4': 0.47045868490430665, 'w5': 0.15365394284719064, 'w6': 0.839568114986856, 'w7': 0.32429242987233436, 'w8': 0.8222821821357059, 'w9': 0.44030363982942655, 'w10': 0.4084038956905105}. Best is trial 40 with value: 0.6998353600502014.
[I 2023-07-22 19:27:03,740] Trial 42 finished with value: 0.6995717883110046 and parameters: {'w1': 0.11178006991299035, 'w2': 0.003405661885284239, 'w3': 0.8134331097841334, 'w4': 0.4654592758160485, 'w5': 0.14336000355577486, 'w6': 0.8402963355071145, 'w7': 0.3190563269059792, 'w8': 0.9224663476421949, 'w9': 0.42691216825957135, 'w10': 0.3902911516999773}. Best is trial 40 with value: 0.6998353600502014.
[I 2023-07-22 19:27:03,822] Trial 43 finished with value: 0.6994029879570007 and parameters: {'w1': 0.06457039415694596, 'w2': 0.08081539394814553, 'w3': 0.8269448872350806, 'w4':

[I 2023-07-22 19:27:05,382] Trial 62 finished with value: 0.699661910533905 and parameters: {'w1': 0.08245522995174513, 'w2': 0.0746589416087895, 'w3': 0.7870326994797946, 'w4': 0.35641123744180664, 'w5': 0.28861075442917294, 'w6': 0.8830199007379682, 'w7': 0.28409191585623733, 'w8': 0.9450562107346019, 'w9': 0.48630199118123346, 'w10': 0.30866388020673735}. Best is trial 52 with value: 0.6999155282974243.
[I 2023-07-22 19:27:05,465] Trial 63 finished with value: 0.6998478174209595 and parameters: {'w1': 0.04342764800160943, 'w2': 0.12613164306451996, 'w3': 0.839301504827335, 'w4': 0.4095068563728392, 'w5': 0.22622264726756913, 'w6': 0.9434171760738491, 'w7': 0.3587286821552005, 'w8': 0.9672442042664056, 'w9': 0.6073608057731348, 'w10': 0.37150233886486045}. Best is trial 52 with value: 0.6999155282974243.
[I 2023-07-22 19:27:05,549] Trial 64 finished with value: 0.6994089484214783 and parameters: {'w1': 0.154448741564835, 'w2': 0.21276484462967982, 'w3': 0.7337364601958285, 'w4': 0.23

[I 2023-07-22 19:27:07,143] Trial 83 finished with value: 0.6998426914215088 and parameters: {'w1': 0.1407055667506379, 'w2': 0.08630534849361543, 'w3': 0.6560795379416922, 'w4': 0.4829903097831244, 'w5': 0.2353930605506158, 'w6': 0.8689012952868673, 'w7': 0.4317351286955584, 'w8': 0.9064006938940385, 'w9': 0.5205623880143172, 'w10': 0.4018916721881831}. Best is trial 68 with value: 0.6999449133872986.
[I 2023-07-22 19:27:07,228] Trial 84 finished with value: 0.6997384428977966 and parameters: {'w1': 0.1704930888969923, 'w2': 0.10797702885916996, 'w3': 0.7126814630209769, 'w4': 0.49382783741602077, 'w5': 0.24787743699495932, 'w6': 0.8777115622550522, 'w7': 0.3912323636535917, 'w8': 0.9042919432305148, 'w9': 0.5144785449488237, 'w10': 0.4082972573137922}. Best is trial 68 with value: 0.6999449133872986.
[I 2023-07-22 19:27:07,312] Trial 85 finished with value: 0.6998546719551086 and parameters: {'w1': 0.13853035156395507, 'w2': 0.02700521247946143, 'w3': 0.6651770773677703, 'w4': 0.5212

...

[I 2023-07-22 19:27:37,915] Trial 414 finished with value: 0.6985703706741333 and parameters: {'w1': 0.41160058128487487, 'w2': 0.08037549318994967, 'w3': 0.9760104272205942, 'w4': 0.4819294321758333, 'w5': 0.2871850810022156, 'w6': 0.5516750305022261, 'w7': 0.8547058315127981, 'w8': 0.7274481211688713, 'w9': 0.527700413232542, 'w10': 0.39996134371288694}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:38,019] Trial 415 finished with value: 0.6983813047409058 and parameters: {'w1': 0.2718403810082534, 'w2': 0.4657593633806454, 'w3': 0.9028301803739841, 'w4': 0.3869510067190998, 'w5': 0.3619706025212804, 'w6': 0.9231326659705004, 'w7': 0.6592095961424944, 'w8': 0.9069319249660122, 'w9': 0.5619445420129912, 'w10': 0.5088545465187885}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:38,123] Trial 416 finished with value: 0.6987289786338806 and parameters: {'w1': 0.061383992568971604, 'w2': 0.02131953557090348, 'w3': 0.8042435858296375, 'w4': 0.

[I 2023-07-22 19:27:40,000] Trial 434 finished with value: 0.6994935870170593 and parameters: {'w1': 0.07738873182955003, 'w2': 0.052922367335229994, 'w3': 0.9873640060650699, 'w4': 0.45294003325837157, 'w5': 0.4693570912525628, 'w6': 0.8516979438004412, 'w7': 0.6681437029150523, 'w8': 0.9558002623882234, 'w9': 0.585693193042854, 'w10': 0.3172347364836483}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:40,105] Trial 435 finished with value: 0.6979745030403137 and parameters: {'w1': 0.04944631419082713, 'w2': 0.6423190124887952, 'w3': 0.9074826275253826, 'w4': 0.5228354372887517, 'w5': 0.338682577568795, 'w6': 0.8828210358778765, 'w7': 0.5778625273710088, 'w8': 0.7461331770177675, 'w9': 0.5542401534510423, 'w10': 0.47758263709697585}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:40,210] Trial 436 finished with value: 0.699867308139801 and parameters: {'w1': 0.0007878146965290109, 'w2': 0.016668264346396545, 'w3': 0.8471194324135612, 'w4':

[I 2023-07-22 19:27:42,222] Trial 455 finished with value: 0.6995557546615601 and parameters: {'w1': 0.03659393070200734, 'w2': 0.09835186512836791, 'w3': 0.8734563192492901, 'w4': 0.5082563819571218, 'w5': 0.3477723403417432, 'w6': 0.617330883543147, 'w7': 0.3795689227301333, 'w8': 0.9636539825753428, 'w9': 0.8205553113322339, 'w10': 0.46211338770204785}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:42,329] Trial 456 finished with value: 0.6987122893333435 and parameters: {'w1': 0.06692267248814796, 'w2': 0.05722585950680796, 'w3': 0.81431069473621, 'w4': 0.5367187327401695, 'w5': 0.32299050954092, 'w6': 0.16238824687226705, 'w7': 0.6480464226427137, 'w8': 0.4966191947649862, 'w9': 0.5516844238312688, 'w10': 0.7825739620450858}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:42,435] Trial 457 finished with value: 0.6981446146965027 and parameters: {'w1': 0.01554540465526845, 'w2': 0.2287354827637783, 'w3': 0.9996819745629115, 'w4': 0.411

[I 2023-07-22 19:27:44,357] Trial 475 finished with value: 0.6991689205169678 and parameters: {'w1': 0.03285934956556574, 'w2': 0.03538549079298095, 'w3': 0.8697053512056258, 'w4': 0.4722060824815226, 'w5': 0.5243365257303094, 'w6': 0.9422054019607493, 'w7': 0.299925625868488, 'w8': 0.8612773441200202, 'w9': 0.6936695304628091, 'w10': 0.8623146954402812}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:44,465] Trial 476 finished with value: 0.6996105313301086 and parameters: {'w1': 0.051244852812384875, 'w2': 0.10706792436875968, 'w3': 0.896200271672753, 'w4': 0.5033144664154882, 'w5': 0.278040357869198, 'w6': 0.959608995238576, 'w7': 0.6327161881284522, 'w8': 0.8355603600475439, 'w9': 0.9400260194624223, 'w10': 0.481566014977184}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:44,573] Trial 477 finished with value: 0.6977443695068359 and parameters: {'w1': 0.3014547657840124, 'w2': 0.7529868565891142, 'w3': 0.4425192917103672, 'w4': 0.27964

[I 2023-07-22 19:27:46,638] Trial 496 finished with value: 0.6988630294799805 and parameters: {'w1': 0.20634884519159724, 'w2': 0.3202080836806398, 'w3': 0.8041269741434911, 'w4': 0.5856603367075716, 'w5': 0.3683447003464913, 'w6': 0.8243008349071678, 'w7': 0.5373033924350549, 'w8': 0.7112741269334795, 'w9': 0.6455416914511292, 'w10': 0.46865327267624785}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:46,747] Trial 497 finished with value: 0.6995014548301697 and parameters: {'w1': 0.03949852935027025, 'w2': 0.0491856807117331, 'w3': 0.834011722623497, 'w4': 0.4050022447693956, 'w5': 0.3986627333810174, 'w6': 0.8953320162675459, 'w7': 0.4916233680521632, 'w8': 0.8590670496028293, 'w9': 0.5773766178619129, 'w10': 0.8065699305429708}. Best is trial 173 with value: 0.7001276016235352.
[I 2023-07-22 19:27:46,856] Trial 498 finished with value: 0.6957007646560669 and parameters: {'w1': 0.08289839600296031, 'w2': 0.5194173084679292, 'w3': 0.7158385700014721, 'w4': 0.88

In [49]:
len(final_preds) 

# Trial 339 finished with value: 0.7002571225166321 and parameters: {'w1': 0.731376155208323, 'w2': 0.4400923709461769, 'w3': 0.5164060638149837, 'w4': 0.4760551206878485, 'w5': 0.4960268099028265, 'w6': 0.96619623095903, 'w7': 0.6882914000281211, 'w8': 0.5731041291145992}. 
best_params # 0.7003294229507446

{'w1': 0.7343510940690922,
 'w2': 0.40892991130135214,
 'w3': 0.4940789919743374,
 'w4': 0.3735780589541978,
 'w5': 0.5516968345932801,
 'w6': 0.8855743368729754,
 'w7': 0.7853496731226426,
 'w8': 0.5825910456327493}

In [ ]:
# 0.37

# {'w1': 0.839014852140699, 'w2': 0.5011350487644706, 'w3': 0.5425134900557348, 'w4': 0.5452545452276221, 'w5': 0.6026139561761139, 'w6': 0.9831197346233449, 'w7': 0.8036481597462362, 'w8': 0.5819171235801198}
# 0.7003488540649414


# Trial 1 finished with value: 0.6991791129112244 and parameters: {'w1': 0.2047525215621725}. Best is trial 1 with value:
# 0.0.6996479034423828
# 0.36
# {
# 'w1': 0.36485377740463365,
# 'w2': 0.5716506827395362,
# 'w3': 0.84301760289886,
# 'w4': 0.5944703175254541,
# 'w5': 0.8096022495041699
# }. Best is trial 202 with value: 0.698759913444519.

In [38]:
{
    'w1': 0.7458107541498612,
    'w2': 0.2900793805943048,
    'w3': 0.40430935291387626,
    'w4': 0.12087058264990641,
    'w5': 0.5101955071950133,
    'w6': 0.28429946034177606,
    'w7': 0.364869611492901,
    'w8': 0.10441135348615381,
    'w9': 0.48784730233361917,
    'w10': 0.9105341528815705
}

 # . Best is trial 289 with value: 0.6934514045715332.


# roh_01_tta roh_02_tta, roh_07_tta, roh_06, roh_07, 
# roh_03_tta, roh_06_tta, roh_03, roh_05, nir_06_tta, nir_01, nir_02
# roh_04_tta, roh_01, nir_02_tta, 131
# roh_02, nir_03_tta, nir_04_tta, nir_05_tta, nir_06, 160
# roh_04, 166
# nir_04, 160_tta
# nir_03, 127

{'w1': 0.7458107541498612,
 'w2': 0.2900793805943048,
 'w3': 0.40430935291387626,
 'w4': 0.12087058264990641,
 'w5': 0.5101955071950133,
 'w6': 0.28429946034177606,
 'w7': 0.364869611492901,
 'w8': 0.10441135348615381,
 'w9': 0.48784730233361917,
 'w10': 0.9105341528815705}

In [44]:
for idx, val in enumerate(call_signs1):
    print(idx+1, val)

1 nir_01_tta
2 nir_05
3 121
4 122
5 125
6 162
7 162_tta
8 166_tta
9 121_tta


In [ ]:
# Trial 145 finished with value: 0.6954816579818726 and parameters: 
# {'w1': 0.3800712934060794, 'w2': 0.19827057881403873, 'w3': 0.8639190435791337, 'w4': 0.8183765509720466, 'w5': 0.17899313642679757, 'w6': 0.024590770644496746, 'w7': 0.7862484916619368, 'w8': 0.992834740398046, 'w9': 0.3995791428881191}. 
# Best is trial 145 with value: 0.6954816579818726. # 0.4

In [ ]:
# [I 2023-07-16 19:49:38,527] Trial 258 finished with value: 0.6955569982528687 and parameters: 
# {'w1': 0.35630544690551724, 'w2': 0.3882333167896021, 'w3': 0.793029020160463, 'w4': 0.589733873322586, 'w5': 0.1238533606039253, 'w6': 0.3136224869358406, 'w7': 0.7445672749651168, 'w8': 0.9585857088778065, 'w9': 0.3523881046014743}. Best is trial 258 with value: 0.6955569982528687.
# 0.41

In [ ]:
# [I 2023-07-16 20:01:35,463] Trial 231 finished with value: 0.6953996419906616 and parameters: {'w1': 0.03615687143100031, 'w2': 0.45224010503148204, 'w3': 0.663721473024527, 'w4': 0.6371224751418889, 'w5': 0.03698106916893698, 'w6': 0.15052361112796397, 'w7': 0.5701940304803765, 'w8': 0.910815487921195, 'w9': 0.47052561326641523}. Best is trial 231 with value: 0.6953996419906616.
# 0.42

In [63]:
# 0.657336 0.01


# call_signs1 = [
#     "roh_01_tta", "nir_01_tta", "nir_02_tta", "nir_03_tta", "nir_04_tta"
# ] 

# call_signs1 = [
#     "roh_02_tta",  "roh_03_tta", "roh_04_tta", "roh_05_tta", "roh_06_tta",  "roh_07_tta",  "roh_08_tta"  
# ] 

call_signs1 = [
#     "roh_01_tta",
#     "roh_02_tta", 
#     "roh_03_tta", 
    
    "roh_04_tta", 
    "roh_05_tta", 
    
    
    
    "nir_01_tta",
#     "nir_02_tta",
#     "nir_03_tta",
#     "nir_04_tta",
    "nir_05_tta",
    
    
    "121", 
    "122",
    "125", 
    "127",
    "131"
    
#     "ioa_01", 
#     "ioa_02",
#     "roh_02_tta"
    
] 

model_masks = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.pt")


preds1 = []

for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    wt = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.pt")    
    preds1.append(wt)
    
    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
    
    
# preds1.append(preds2)
    
final_preds = preds1
final_preds = torch.stack(final_preds).mean(dim=0)
score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 

print("0.5 TH Score: ", score)


best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] :
    score = dice_coef(model_masks, final_preds, thr=threshold).cpu().detach().numpy() 
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold
print(best_dice_score, best_threshold)

FileNotFoundError: [Errno 2] No such file or directory: '/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.pt'

In [39]:
# 0.5 TH Score:  0.67887855
# 0.6856482 0.28



# 0.5 TH Score:  0.6811164
# 0.6867434 0.35


In [61]:
def objective(trial):
    TH = None
    
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1) 
        
    score, best_threshold  = boost_dice(params, TH=TH)
    params['threshold'] = best_threshold
    if not TH:
        print(params)
        
    return score

In [62]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)
best_params = study.best_params

[I 2023-07-18 13:31:08,096] A new study created in memory with name: no-name-a2e003a6-8cca-4003-ad7f-564093208279
[W 2023-07-18 13:31:18,130] Trial 0 failed with parameters: {'w1': 0.7634220896775001, 'w2': 0.03848726673023917, 'w3': 0.05673113846045508, 'w4': 0.8288959020652308, 'w5': 0.6297679994975424, 'w6': 0.13844310031440676, 'w7': 0.29140066140564935, 'w8': 0.37732839539816343, 'w9': 0.28599952944558493, 'w10': 0.38562275625224585, 'w11': 0.04134742892497112, 'w12': 0.20397317316305197, 'w13': 0.27369480824659587, 'w14': 0.16733076141469339, 'w15': 0.315575152111762, 'w16': 0.2663270869274461, 'w17': 0.7143205059493493, 'w18': 0.47297724520132145, 'w19': 0.11424872433172839, 'w20': 0.33568452865166376, 'w21': 0.34618702919261757, 'w22': 0.6073129554981073, 'w23': 0.38706744563454676, 'w24': 0.572389191361773, 'w25': 0.3441876700651719, 'w26': 0.07675091883675134, 'w27': 0.013784393918564453, 'w28': 0.5068739383633645, 'w29': 0.8078668700887537, 'w30': 0.35311902669396644, 'w31':

[W 2023-07-18 13:31:18,133] Trial 0 failed with value None.


KeyboardInterrupt: 

In [ ]:
for idx, val in enumerate(call_signs1):
    print(idx+1, val)

In [34]:
call_signs1 = [
    "roh_01_tta_full", "nir_01_tta_full", "nir_02_tta_full", "nir_03_tta_full", "nir_04_tta_full"
] 

model_masks = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks_full.pt")


preds1 = []

for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    wt = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.pt")    
    preds1.append(wt)
    
    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
    
final_preds = preds1
final_preds = torch.stack(final_preds).mean(dim=0)
score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 
print("0.5 TH Score: ", score)

best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] :
    score = dice_coef(model_masks, final_preds, thr=threshold).cpu().detach().numpy() 
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold
print(best_dice_score, best_threshold)


  0%|          | 0/5 [00:00<?, ?it/s]

0.6469362
0.6567023
0.6457618
0.6704817
0.655622
0.5 TH Score:  0.6751985
0.6808599 0.35


In [ ]:
# 0.5 TH Score:  0.6760312
# 0.6810049 0.33

In [ ]:
# CFGS1 = [   
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'model_func': load_model_exp28,
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#         'call_sign': "ioa_01",
#         'tta': True, #False, #True
#         'normalize': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "ioa_02",
#         'model_func': load_model_exp28_snapshot,
#         'tta': False, #True
#         'normalize': True,
#         'model_pth': [
#             ######## full train.csv 
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#             ######### 4-skf split 
#         ]
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "roh_02",
#         'model_func': load_model_exp28_snapshot,        
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#         'tta': False, #True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "roh_03",
#         'model_func': load_model_exp28_snapshot,        
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#         'tta': False, #True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "roh_04",
#         'model_func': load_model_exp28_snapshot,        
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#         'tta': False, #True
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "ioa_02",
#         'model_func': load_model_exp28_snapshot,
#         'tta': False, #True
#         'normalize': True,
#         'model_pth': [
#             ######## full train.csv 
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#             ######### 4-skf split 
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v1.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v2.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v3.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v4.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v5.ckpt'
#         ]
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "ioa_02_tta",
#         'model_func': load_model_exp28_snapshot,
#         'tta': True, #True
#         'normalize': True,
#         'model_pth': [
#             ######## full train.csv 
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#             ######### 4-skf split 
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v1.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v2.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v3.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v4.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v5.ckpt'
#         ]
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_tta",
#         'model_func': load_modelr1,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01",
#         'model_func': load_modelr1,
#         'tta': False
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'cs3darknet_l',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/cs3darknet_l-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_02_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'cs3edgenet_x',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/cs3edgenet_x-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_03_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'cs3sedarknet_l',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/cs3sedarknet_l-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_04_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'ecaresnet101d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/ecaresnet101d-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_05_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'ecaresnet269d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/ecaresnet269d-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_06_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'fbnetv3_g',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/fbnetv3_g-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_07_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
        
#     {
#         'model_name': 'Unet',
#         'backbone': 'tf_mixnet_l',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/tf_mixnet_l-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_08_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
    

#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01_tta",
#         'model_func': load_model4,
#         'tta': True
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02_tta",
#         'model_func': load_model2,
#         'tta': True
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03_tta", 
#         'model_func': load_model3,
#         'tta': True
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04_tta", 
#         'model_func': load_model1,
#         'tta': True
#     }, 
    
    
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l2',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_05_tta",
#         'model_func': load_model2,
#         'tta': True
#     },
#     {
#     'model_name': 'Unet++',
#         'backbone': 'densenetblur121d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_densenetblur121d_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_06_tta", 
#         'model_func': load_model2,
#         'tta': True
#     },
    
#     {
#     'model_name': 'Unet++',
#         'backbone': 'convnextv2_base',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07_tta", 
#         'model_func': load_model2,
#         'tta': True
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01",
#         'model_func': load_model4,
#         'tta': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02",
#         'model_func': load_model2,
#         'tta': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03", 
#         'model_func': load_model3,
#         'tta': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04", 
#         'model_func': load_model1,
#         'tta': False
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l2',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_05",
#         'model_func': load_model2,
#         'tta': False
#     },
#     {
#     'model_name': 'Unet++',
#         'backbone': 'densenetblur121d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_densenetblur121d_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_06", 
#         'model_func': load_model2,
#         'tta': False
#     },
    
#     {
#     'model_name': 'Unet++',
#         'backbone': 'convnextv2_base',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07", 
#         'model_func': load_model2,
#         'tta': False
#     },
    
    
    
    
    
# ]